In [ ]:
!pip install tensorflow kaggle pillow matplotlib tensorflowjs keras-tuner

In [ ]:
import os
import random
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflowjs as tfjs
from keras_tuner import RandomSearch, Hyperband, BayesianOptimization

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

train_directory = "/kaggle/input/nepali-food-images/dataset/train"
test_directory = "/kaggle/input/nepali-food-images/dataset/test"

batch_size, img_width, img_height = 32, 224, 224

In [ ]:
def count_files_and_images(dir):
    class_count = {}
    total = 0
    for class_name in os.listdir(dir):
        path = os.path.join(dir, class_name)
        if os.path.isdir(path):
            file_count = len(os.listdir(path))
            class_count[class_name] = file_count
            total += file_count
    return class_count, total

In [ ]:
train_class_count, train_total = count_files_and_images(train_directory)
test_class_count, test_total = count_files_and_images(test_directory)

print("Train class count: ", train_class_count)
print("Train total: ", train_total)

print("Test class count: ", test_class_count)
print("Test total: ", test_total)

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    shuffle=True,
    batch_size=batch_size,
    labels="inferred",
)
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_directory,
    batch_size=batch_size,
    labels="inferred",
)
class_names = train_dataset.class_names
print("Class names: ", class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_dataset.class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def show_history(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(len(loss))

  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()

In [ ]:
def fit_model(model, epochs, model_name):
  history = model.fit(train_dataset,
                        validation_data=test_dataset,
                        epochs=epochs,
                        callbacks=[
                            ModelCheckpoint(
                                filepath=f'/kaggle/temp/weights/{model_name}',
                                save_weights_only=True,
                                verbose=1,
                                save_freq='epoch',
                                period=20),
                            EarlyStopping(
                                monitor='val_loss', 
                                min_delta=0.001, 
                                patience=10, 
                                mode='min', 
                                restore_best_weights=True
                            )
                        ])
  # Save the model in the Keras format
  model_path = f'/kaggle/working/saved/{model_name}.keras'
  model.save(model_path, overwrite=True, save_format="keras")
  print(f"Model saved to {model_path}")

  # Convert the Keras model to TensorFlow.js format
  tfjs_target_dir = f'/kaggle/working/saved/{model_name}_tfjs'
  tfjs.converters.save_keras_model(model, tfjs_target_dir)
  print(f"Model converted to TensorFlow.js format and saved to {tfjs_target_dir}")

  model.summary()
  show_history(history)

In [ ]:
def simple_multilayer_model():
    model = Sequential([
      layers.Resizing(img_width, img_height),
      layers.Rescaling(1./255,input_shape=(180, 180, 3)),
      layers.Flatten(),
      layers.Dense(2000, activation='relu'),
      layers.Dense(1000, activation='relu'),
      layers.Dense(500, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(len(class_names), name="outputs")
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    epochs = 100
    fit_model(model, epochs, 'simple_multilayer')

simple_multilayer_model()

In [ ]:
def augmented_multilayer_model():
    model = Sequential([
      layers.Resizing(img_width, img_height),
      layers.Rescaling(1./255),
      layers.RandomFlip(),
      layers.RandomRotation(.2),
      layers.RandomZoom(.2),
      layers.Flatten(),
      layers.Dense(2000, activation='relu'),
      layers.Dense(1000, activation='relu'),
      layers.Dropout(0.25),
      layers.Dense(500, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.05),
      layers.Dense(len(class_names), name="outputs")
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    epochs = 100
    fit_model(model, epochs, 'augmented_multilayer')

augmented_multilayer_model()

In [ ]:
def augmented_conv_model():
    model = Sequential([
      layers.Resizing(img_width, img_height),
      layers.Rescaling(1./255),
      layers.RandomFlip(),
      layers.RandomRotation(.2),
      layers.RandomZoom(.2),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(512, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(128, activation='relu'),
      layers.Dense(len(class_names), name="outputs")
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    epochs = 100
    fit_model(model, epochs, 'augmented_conv')

augmented_conv_model()

In [ ]:
def augmented_conv_model_v2():
    model = Sequential([
      layers.Resizing(img_width, img_height),
      layers.Rescaling(1./255),
      layers.RandomFlip(),
      layers.RandomRotation(.1),
      layers.RandomZoom(.1),
      layers.RandomTranslation(0.1, 0.1),
      layers.RandomContrast(0.1),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.BatchNormalization(),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.BatchNormalization(),
      layers.MaxPooling2D(),
      layers.Conv2D(128, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(64, activation='relu'),
      layers.Dropout(0.05),
      layers.Dense(len(class_names), name="outputs")
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    epochs = 100
    fit_model(model, epochs, 'augmented_conv')

augmented_conv_model_v2()

In [ ]:
def hyper_parameter_optimized_augmented_conv_model():
    def build_model(hp):
        model = keras.Sequential([
            layers.Resizing(img_width, img_height),
            layers.Rescaling(1./255),
            layers.RandomFlip(),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.RandomTranslation(0.1, 0.1),
            layers.RandomContrast(0.1),
            layers.Conv2D(
                filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
                kernel_size=3,
                padding='same',
                activation='relu'
            ),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(
                filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=32),
                kernel_size=3,
                padding='same',
                activation='relu'
            ),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(
                filters=hp.Int('conv_3_filters', min_value=128, max_value=512, step=32),
                kernel_size=3,
                padding='same',
                activation='relu'
            ),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(
                units=hp.Int('dense_units', min_value=32, max_value=128, step=32),
                activation='relu'
            ),
            layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.01)),
            layers.Dense(len(class_names), name="outputs")
        ])

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            ),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

        return model

    tuner = Hyperband(
        build_model,
        objective='val_accuracy',
        max_epochs=10,
        factor=3,
        directory='tuning',
        project_name='image_classification_tuning'
    )
    
    tuner.search(train_dataset, epochs=10, validation_data=test_dataset)
    print("found")
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(best_hps)
    # Build the model with the optimal hyperparameters and train it on the data
    model = tuner.hypermodel.build(best_hps)
    
    epochs = 100
    fit_model(model, epochs, 'augmented_conv')

hyper_parameter_optimized_augmented_conv_model()

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model

def fine_tune_efficient_net():
    # Load the pre-trained model, excluding the top layer
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False
    
    # Add our new input layer and output layer to the base model
    model = Sequential([
        layers.InputLayer(input_shape=(None, None, 3)),
        layers.Resizing(224, 224),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(len(class_names), activation='softmax')
    ])
    
    epochs = 10
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )
    fit_model(model, epochs, 'efficient_net_finetune')

fine_tune_efficient_net()

In [ ]:
!rm -rf /kaggle/working/saved/efficient_net_finetune_tfjs
!pip install numpy==1.22.4

In [ ]:
!zip -r /kaggle/working/saved/efficient_net_finetune_tfjs.zip /kaggle/working/saved/efficient_net_finetune_tfjs